In [1]:
from evo import Evo

evo_model = Evo('evo-1-131k-base')
test = print("dd")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

dd


In [3]:
%%bash
set -e  # stop on error

echo "Création des dossiers..."
mkdir -p datasets/refseq_simulated_metagenome
mkdir -p datasets/deephage_lifestyle

echo "Téléchargement Dataset 3 (refseq_simulated_metagenome)..."
wget -q -O datasets/refseq_simulated_metagenome.tar.gz https://phagos-rd-hackathon25-datasets.s3.eu-west-1.amazonaws.com/refseq_simulated_metagenome/refseq_simulated_metagenome.tar.gz
echo "Extraction Dataset 3..."
tar -xzf datasets/refseq_simulated_metagenome.tar.gz -C datasets/refseq_simulated_metagenome
rm datasets/refseq_simulated_metagenome.tar.gz
echo "Dataset 3 prêt."

echo "Téléchargement Dataset 4 (deephage_lifestyle)..."
wget -q -O datasets/deephage_lifestyle.tar.gz https://s3.eu-west-1.amazonaws.com/phagos-rd-hackathon25-datasets/deephage/deephage_lifestyle.tar.gz
echo "Extraction Dataset 4..."
tar -xzf datasets/deephage_lifestyle.tar.gz -C datasets/deephage_lifestyle
rm datasets/deephage_lifestyle.tar.gz
echo "Dataset 4 prêt."

echo "Tous les datasets ont été installés."


Création des dossiers...
Téléchargement Dataset 3 (refseq_simulated_metagenome)...
Extraction Dataset 3...
Dataset 3 prêt.
Téléchargement Dataset 4 (deephage_lifestyle)...
Extraction Dataset 4...
Dataset 4 prêt.
Tous les datasets ont été installés.
